In [2]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
from scipy.spatial import ConvexHull

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

/var/folders/6s/v2ptr27x1537n4_9dh_x4n0h0000gq/T/ipykernel_55889/3409617640.py:24: DeprecationWarning: Please use `label` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  from scipy.ndimage.measurements import label


In [ ]:
yrs=np.arange(1980,2020,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir2+'era5.ar.tempestlr'+str(year)+'.nc')
    ar=ds['ar']
    ar=ar.where(ar==1)
    freq_era5tp=ar.sum(dim='time',skipna=True)
    
    ds=xr.open_dataset(dir2+'era5.ar.reid500.'+str(year)+'.nc')
    ar=ds['ar']
    ar=ar.where(ar==1)
    freq_era5reid=ar.sum(dim='time',skipna=True)
    
    ds=xr.open_dataset(dir2+'era5.ar.mundhenkv3.'+str(year)+'.nc')
    ar=ds['ar']
    ar=ar.where(ar==1)
    freq_era5mk=ar.sum(dim='time',skipna=True)
    
    ds=xr.open_dataset(dir2+'merra2.ar.lorav2.'+str(year)+'.nc')
    ar=ds['ar']
    ar=ar.where(ar==1)
    freq_merra2lo=ar.sum(dim='time',skipna=True)
    
    ds=xr.open_dataset(dir2+'merra2.ar.cndl.'+str(year)+'.nc')
    ar=ds['ar']
    ar=ar.where(ar==1)
    freq_merra2cndl=ar.sum(dim='time',skipna=True)
    
    ds=xr.open_dataset(dir2+'merra2.ar.acv2.'+str(year)+'.nc')
    ar=ds['ar']
    ar=ar.where(ar==1)
    freq_merra2ac=ar.sum(dim='time',skipna=True)
    
    ds=xr.open_dataset(dir2+'jra55.ar.gwv2.'+str(year)+'.nc')
    ar=ds['ar']
    ar=ar.where(ar==1)
    freq_jra55gw=ar.sum(dim='time',skipna=True)
    
    if i==0:
        freq_concat_era5tp=freq_era5tp
        freq_concat_era5reid=freq_era5reid
        freq_concat_era5mk=freq_era5mk
        
        freq_concat_merra2lo=freq_merra2lo
        freq_concat_merra2cndl=freq_merra2cndl
        freq_concat_merra2ac=freq_merra2ac
        
        freq_concat_jra55gw=freq_jra55gw
    else:
        freq_concat_era5tp=xr.concat([freq_concat_era5tp,freq_era5tp],dim='time')
        freq_concat_era5reid=xr.concat([freq_concat_era5reid,freq_era5reid],dim='time')
        freq_concat_era5mk=xr.concat([freq_concat_era5mk,freq_era5mk],dim='time')
        
        freq_concat_merra2lo=xr.concat([freq_concat_merra2lo,freq_merra2lo],dim='time')
        freq_concat_merra2cndl=xr.concat([freq_concat_merra2cndl,freq_merra2cndl],dim='time')
        freq_concat_merra2ac=xr.concat([freq_concat_merra2ac,freq_merra2ac],dim='time')
        
        freq_concat_jra55gw=xr.concat([freq_concat_jra55gw,freq_jra55gw],dim='time')

In [ ]:
lons_west=[x for x in freq_concat_jra55gw.lon.values if x<-90]
lons_east=[x for x in freq_concat_jra55gw.lon.values if x>=90]
sel_west=freq_concat_jra55gw.sel(lon=lons_west)
sel_east=freq_concat_jra55gw.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,freq_concat_jra55gw,sel_west],dim='lon')
interp=final.interp(lat=freq_concat_era5tp.lat.values,lon=freq_concat_era5tp.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres=interp.sel(lon=lons_center)

In [ ]:
slopes_array_era5tp=xr.zeros_like(freq_concat_era5tp[0,:,:])
sigs_array_era5tp=xr.zeros_like(freq_concat_era5tp[0,:,:])

slopes_array_era5reid=xr.zeros_like(freq_concat_era5tp[0,:,:])
sigs_array_era5reid=xr.zeros_like(freq_concat_era5tp[0,:,:])

slopes_array_era5mk=xr.zeros_like(freq_concat_era5tp[0,:,:])
sigs_array_era5mk=xr.zeros_like(freq_concat_era5tp[0,:,:])

slopes_array_merra2lo=xr.zeros_like(freq_concat_era5tp[0,:,:])
sigs_array_merra2lo=xr.zeros_like(freq_concat_era5tp[0,:,:])

slopes_array_merra2cndl=xr.zeros_like(freq_concat_era5tp[0,:,:])
sigs_array_merra2cndl=xr.zeros_like(freq_concat_era5tp[0,:,:])

slopes_array_merra2ac=xr.zeros_like(freq_concat_era5tp[0,:,:])
sigs_array_merra2ac=xr.zeros_like(freq_concat_era5tp[0,:,:])

slopes_array_jra55gw=xr.zeros_like(freq_concat_era5tp[0,:,:])
sigs_array_jra55gw=xr.zeros_like(freq_concat_era5tp[0,:,:])
size=20
for j in range(freq_concat_era5tp.lat.size):
    print(j)
    for k in range(freq_concat_era5tp.lon.size):
        time_series=freq_concat_era5tp[:,j,k].values.tolist()
        defined=[x for x in time_series if x>0]
        r=st.linregress(yrs,time_series)
        if len(defined)>=size:
            slopes_array_era5tp[j,k]=r[0]
            sigs_array_era5tp[j,k]=r[3]
        else:
            slopes_array_era5tp[j,k]=np.nan
            sigs_array_era5tp[j,k]=np.nan
            
            
        time_series=freq_concat_era5reid[:,j,k].values.tolist()
        defined=[x for x in time_series if x>0]
        r=st.linregress(yrs,time_series)
        if len(defined)>=size:
            slopes_array_era5reid[j,k]=r[0]
            sigs_array_era5reid[j,k]=r[3]
        else:
            slopes_array_era5reid[j,k]=np.nan
            sigs_array_era5reid[j,k]=np.nan
            
        
        time_series=freq_concat_era5mk[:,j,k].values.tolist()
        defined=[x for x in time_series if x>0]
        r=st.linregress(yrs,time_series)
        if len(defined)>=size:
            slopes_array_era5mk[j,k]=r[0]
            sigs_array_era5mk[j,k]=r[3]
        else:
            slopes_array_era5mk[j,k]=np.nan
            sigs_array_era5mk[j,k]=np.nan
            
            
        time_series=freq_concat_merra2lo[:,j,k].values.tolist()
        defined=[x for x in time_series if x>0]
        r=st.linregress(yrs,time_series)
        if len(defined)>=size:
            slopes_array_merra2lo[j,k]=r[0]
            sigs_array_merra2lo[j,k]=r[3]
        else:
            slopes_array_merra2lo[j,k]=np.nan
            sigs_array_merra2lo[j,k]=np.nan
            
            
        time_series=freq_concat_merra2cndl[:,j,k].values.tolist()
        defined=[x for x in time_series if x>0]
        r=st.linregress(yrs,time_series)
        if len(defined)>=size:
            slopes_array_merra2cndl[j,k]=r[0]
            sigs_array_merra2cndl[j,k]=r[3]
        else:
            slopes_array_merra2cndl[j,k]=np.nan
            sigs_array_merra2cndl[j,k]=np.nan
            
            
        time_series=freq_concat_merra2ac[:,j,k].values.tolist()
        defined=[x for x in time_series if x>0]
        r=st.linregress(yrs,time_series)
        if len(defined)>=size:
            slopes_array_merra2ac[j,k]=r[0]
            sigs_array_merra2ac[j,k]=r[3]
        else:
            slopes_array_merra2ac[j,k]=np.nan
            sigs_array_merra2ac[j,k]=np.nan
            
            
        time_series=fullres[:,j,k].values.tolist()
        defined=[x for x in time_series if x>0]
        r=st.linregress(yrs,time_series)
        if len(defined)>=size:
            slopes_array_jra55gw[j,k]=r[0]
            sigs_array_jra55gw[j,k]=r[3]
        else:
            slopes_array_jra55gw[j,k]=np.nan
            sigs_array_jra55gw[j,k]=np.nan

In [ ]:
dk=xr.Dataset()
dk['slopes_array_era5tp']=(('lat','lon'),slopes_array_era5tp.values)
dk['slopes_array_era5reid']=(('lat','lon'),slopes_array_era5reid.values)
dk['slopes_array_era5mk']=(('lat','lon'),slopes_array_era5mk.values)

dk['slopes_array_merra2lo']=(('lat','lon'),slopes_array_merra2lo.values)
dk['slopes_array_merra2cndl']=(('lat','lon'),slopes_array_merra2cndl.values)
dk['slopes_array_merra2ac']=(('lat','lon'),slopes_array_merra2ac.values)

dk['slopes_array_jra55gw']=(('lat','lon'),slopes_array_jra55gw.values)

dk['sigs_array_era5tp']=(('lat','lon'),sigs_array_era5tp.values)
dk['sigs_array_era5reid']=(('lat','lon'),sigs_array_era5reid.values)
dk['sigs_array_era5mk']=(('lat','lon'),sigs_array_era5mk.values)

dk['sigs_array_merra2lo']=(('lat','lon'),sigs_array_merra2lo.values)
dk['sigs_array_merra2cndl']=(('lat','lon'),sigs_array_merra2cndl.values)
dk['sigs_array_merra2ac']=(('lat','lon'),sigs_array_merra2ac.values)

dk['sigs_array_jra55gw']=(('lat','lon'),sigs_array_jra55gw.values)

dk.coords['lat']=slopes_array_era5tp.lat.values
dk.coords['lon']=slopes_array_era5tp.lon.values

dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'

try:
    os.remove(dir_data+'ar_geoslopes_artmip.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_geoslopes_artmip.nc',mode='w',format='NETCDF4')

In [ ]:
p_values=sigs_array_era5reid.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=0.1*index/float(N):
        pass_list.append(p_value)
try:
    p_final_era5reid=max(pass_list)
    print(p_final_era5reid)
except:
    p_final_era5reid=np.nan
    print(p_final_era5reid)

p_values=sigs_array_era5tp.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=0.1*index/float(N):
        pass_list.append(p_value)
try:
    p_final_era5tp=max(pass_list)
    print(p_final_era5tp)
except:
    p_final_era5tp=np.nan
    print(p_final_era5tp)

p_values=sigs_array_era5mk.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=0.1*index/float(N):
        pass_list.append(p_value)
try:
    p_final_era5mk=max(pass_list)
    print(p_final_era5mk)
except:
    p_final_era5mk=np.nan
    print(p_final_era5mk)


p_values=sigs_array_merra2lo.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=0.1*index/float(N):
        pass_list.append(p_value)
try:
    p_final_merra2lo=max(pass_list)
    print(p_final_merra2lo)
except:
    p_final_merra2lo=np.nan
    print(p_final_merra2lo)

p_values=sigs_array_merra2cndl.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=0.1*index/float(N):
        pass_list.append(p_value)
try:
    p_final_merra2cndl=max(pass_list)
    print(p_final_merra2cndl)
except:
    p_final_merra2cndl=np.nan
    print(p_final_merra2cndl)
    
p_values=sigs_array_merra2ac.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=0.1*index/float(N):
        pass_list.append(p_value)
try:
    p_final_merra2ac=max(pass_list)
    print(p_final_merra2ac)
except:
    p_final_merra2ac=np.nan
    print(p_final_merra2ac)
    

p_values=sigs_array_jra55gw.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=0.1*index/float(N):
        pass_list.append(p_value)
try:
    p_final_jra55gw=max(pass_list)
    print(p_final_jra55gw)
except:
    p_final_jra55gw=np.nan
    print(p_final_jra55gw)

In [ ]:
field=slopes_array_jra55gw
mean=fullres.mean(dim='time')
field=field/mean*100.
field=field.where(mean>0)
field1=sigs_array_jra55gw

from matplotlib.colors import ListedColormap
cmap1=plt.cm.seismic(np.linspace(0.2,0.8,128))
cmap_new=ListedColormap(cmap1)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.5,vmax=1.5,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=25,ticks=np.arange(-1.5,2,0.5))

#X=6
X=4
for j in range(int(field1.lat.size/X)):
    for k in range(int(field1.lon.size/X)):
        if field1[X*j,X*k].values.tolist()<=0.05:
            ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=31) 
cbar.set_label('% yr$^{-1}$',fontsize=34,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':31,'color':'k'}
g1.ylabel_style={'size':31,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
plt.show()

In [ ]:
num_sets1=xr.ones_like(slopes_array_era5tp).where(slopes_array_era5tp>-1000)
num_sets1=num_sets1.fillna(0)
num_sets2=xr.ones_like(slopes_array_era5reid).where(slopes_array_era5reid>-1000)
num_sets2=num_sets2.fillna(0)
num_sets3=xr.ones_like(slopes_array_era5mk).where(slopes_array_era5mk>-1000)
num_sets3=num_sets3.fillna(0)

num_sets4=xr.ones_like(slopes_array_merra2lo).where(slopes_array_merra2lo>-1000)
num_sets4=num_sets4.fillna(0)
num_sets5=xr.ones_like(slopes_array_merra2cndl).where(slopes_array_merra2cndl>-1000)
num_sets5=num_sets5.fillna(0)
num_sets6=xr.ones_like(slopes_array_merra2ac).where(slopes_array_merra2ac>-1000)
num_sets6=num_sets6.fillna(0)

num_sets7=xr.ones_like(slopes_array_jra55gw).where(slopes_array_jra55gw>-1000)
num_sets7=num_sets7.fillna(0)

num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6+num_sets7
field=num_sets


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,8,1),cmap=plt.cm.Greens,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,8,1))

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('# datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_ylim(-60,70)

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{\#}$'+' '+r'$\bf{datasets:}$'+' '+r'$\bf{ARTMIP}$'+' '+r'$\bf{Tier}$'+' '+r'$\bf{2}$'+' '+r'$\bf{sample}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_redo_1.png')

In [ ]:
# datasets - 
# datasets - 3 AR variants
# datasets - 2 EIVT variants


test1=xr.ones_like(sigs_array_era5tp).where(sigs_array_era5tp<=p_final_era5tp).where(slopes_array_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_era5reid).where(sigs_array_era5reid<=p_final_era5reid).where(slopes_array_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_era5mk).where(sigs_array_era5mk<=p_final_era5mk).where(slopes_array_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_merra2lo).where(sigs_array_merra2lo<=p_final_merra2lo).where(slopes_array_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_merra2cndl).where(sigs_array_merra2cndl<=p_final_merra2cndl).where(slopes_array_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_merra2ac).where(sigs_array_merra2ac<=p_final_merra2ac).where(slopes_array_merra2ac>0).fillna(0)

test7=xr.ones_like(sigs_array_jra55gw).where(sigs_array_jra55gw<=p_final_jra55gw).where(slopes_array_jra55gw>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_era5tp).where(sigs_array_era5tp<=p_final_era5tp).where(slopes_array_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_era5reid).where(sigs_array_era5reid<=p_final_era5reid).where(slopes_array_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_era5mk).where(sigs_array_era5mk<=p_final_era5mk).where(slopes_array_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_merra2lo).where(sigs_array_merra2lo<=p_final_merra2lo).where(slopes_array_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_merra2cndl).where(sigs_array_merra2cndl<=p_final_merra2cndl).where(slopes_array_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_merra2ac).where(sigs_array_merra2ac<=p_final_merra2ac).where(slopes_array_merra2ac<0).fillna(0)

test7a=xr.ones_like(sigs_array_jra55gw).where(sigs_array_jra55gw<=p_final_jra55gw).where(slopes_array_jra55gw<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)


#######

test1b=xr.ones_like(sigs_array_era5tp).where(sigs_array_era5tp<=0.05).where(slopes_array_era5tp>0).fillna(0)
test2b=xr.ones_like(sigs_array_era5reid).where(sigs_array_era5reid<=0.05).where(slopes_array_era5reid>0).fillna(0)
test3b=xr.ones_like(sigs_array_era5mk).where(sigs_array_era5mk<=0.05).where(slopes_array_era5mk>0).fillna(0)

test4b=xr.ones_like(sigs_array_merra2lo).where(sigs_array_merra2lo<=0.05).where(slopes_array_merra2lo>0).fillna(0)
test5b=xr.ones_like(sigs_array_merra2cndl).where(sigs_array_merra2cndl<=0.05).where(slopes_array_merra2cndl>0).fillna(0)
test6b=xr.ones_like(sigs_array_merra2ac).where(sigs_array_merra2ac<=0.05).where(slopes_array_merra2ac>0).fillna(0)

test7b=xr.ones_like(sigs_array_jra55gw).where(sigs_array_jra55gw<=0.05).where(slopes_array_jra55gw>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b+test7b)/num_sets*100.


test1c=xr.ones_like(sigs_array_era5tp).where(sigs_array_era5tp<=0.05).where(slopes_array_era5tp<0).fillna(0)
test2c=xr.ones_like(sigs_array_era5reid).where(sigs_array_era5reid<=0.05).where(slopes_array_era5reid<0).fillna(0)
test3c=xr.ones_like(sigs_array_era5mk).where(sigs_array_era5mk<=0.05).where(slopes_array_era5mk<0).fillna(0)

test4c=xr.ones_like(sigs_array_merra2lo).where(sigs_array_merra2lo<=0.05).where(slopes_array_merra2lo<0).fillna(0)
test5c=xr.ones_like(sigs_array_merra2cndl).where(sigs_array_merra2cndl<=0.05).where(slopes_array_merra2cndl<0).fillna(0)
test6c=xr.ones_like(sigs_array_merra2ac).where(sigs_array_merra2ac<=0.05).where(slopes_array_merra2ac<0).fillna(0)

test7c=xr.ones_like(sigs_array_jra55gw).where(sigs_array_jra55gw<=0.05).where(slopes_array_jra55gw<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c+test7c)/num_sets*100.

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))

cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1.,levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_allb[X*j,X*k]>=66.66 or test_allc[X*j,X*k]>=66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='k',markersize=2,alpha=0.5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{Trends:}$'+' '+r'$\bf{ARTMIP}$'+' '+r'$\bf{Tier}$'+' '+r'$\bf{2}$'+' '+r'$\bf{sample}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_redo_2.png')